In [1]:
import duckdb

# puzzle input goes here
input = """
"""

In [2]:
duckdb.sql("""
    WITH RECURSIVE input AS (
        SELECT '%s' AS input
    ), workflows AS (
        SELECT SUBSTRING(wf, 1, INSTR(wf, '{')-1) wf_name, STRING_SPLIT(REGEXP_EXTRACT(wf, '\\{([^}]*)\\}', 1), ',') wf_route
        FROM (
            SELECT UNNEST(STRING_SPLIT(SPLIT_PART(input, '\n\n', 1), '\n')) wf
            FROM input
        )
    ), parts AS (
        SELECT
            ROW_NUMBER() OVER() part_no,
            CAST(REGEXP_EXTRACT(part, 'x=([0-9]+)', 1) AS INT) AS x,
            CAST(REGEXP_EXTRACT(part, 'm=([0-9]+)', 1) AS INT) AS m,
            CAST(REGEXP_EXTRACT(part, 'a=([0-9]+)', 1) AS INT) AS a,
            CAST(REGEXP_EXTRACT(part, 's=([0-9]+)', 1) AS INT) AS s
        FROM (
            SELECT UNNEST(STRING_SPLIT(SPLIT_PART(input, '\n\n', 2), '\n')) part
            FROM input
        )
    ), routes AS (
        SELECT wf_name, route_idx, SUBSTRING(cond, 1, 1) xmas, SUBSTRING(cond, 2, 1) op, CAST(SUBSTRING(cond, 3) AS INT) val, tgt
        FROM (
            SELECT wf_name, route_idx, NULLIF(REGEXP_EXTRACT(route, '([^:]+):', 1), '') cond, SPLIT_PART(route, ':', -1) tgt
            FROM (
                SELECT wf_name, GENERATE_SUBSCRIPTS(wf_route, 1) AS route_idx, UNNEST(wf_route) route
                FROM workflows
            )
        )
    ), wf_part_check AS (
        SELECT part_no, wf_name, route_idx, x, m, a, s,
            CASE
                WHEN val IS NULL OR
                    CASE
                        WHEN xmas = 'x' THEN x-val
                        WHEN xmas = 'm' THEN m-val
                        WHEN xmas = 'a' THEN a-val
                        WHEN xmas = 's' THEN s-val
                    END *
                    CASE
                        WHEN op = '>' THEN 1
                        ELSE -1 -- there are only > or <
                    END > 0
                THEN tgt
                ELSE wf_name
            END result,
            route_idx + 1 next_route_idx
        FROM parts, routes
        WHERE routes.wf_name = 'in'
        AND routes.route_idx = 1
        UNION ALL
        SELECT wf_part_check.part_no, routes.wf_name, routes.route_idx, wf_part_check.x, wf_part_check.m, wf_part_check.a, wf_part_check.s,
            CASE
                WHEN routes.val IS NULL OR
                    CASE
                        WHEN routes.xmas = 'x' THEN wf_part_check.x-routes.val
                        WHEN routes.xmas = 'm' THEN wf_part_check.m-routes.val
                        WHEN routes.xmas = 'a' THEN wf_part_check.a-routes.val
                        WHEN routes.xmas = 's' THEN wf_part_check.s-routes.val
                    END *
                    CASE
                        WHEN routes.op = '>' THEN 1
                        ELSE -1 -- there are only > or <
                    END > 0
                THEN routes.tgt
                ELSE routes.wf_name
            END result,
            routes.route_idx + 1 next_route_idx
        FROM wf_part_check, routes
        WHERE wf_part_check.result NOT IN ('A', 'R')
        AND wf_part_check.result = routes.wf_name
        AND (
            wf_part_check.result = wf_part_check.wf_name AND wf_part_check.next_route_idx = routes.route_idx
            OR
            wf_part_check.result != wf_part_check.wf_name AND routes.route_idx = 1
        )
    ), check_results AS (
        SELECT COUNT(DISTINCT part_no), SUM(CAST(result='A' AS INT)) result_A, SUM(CAST(result='R' AS INT)) result_R
        FROM wf_part_check
    )
    SELECT SUM(x) + SUM(m) + SUM(a) + SUM(s) total_rating
    FROM wf_part_check
    WHERE result = 'A'
""" % input)

┌──────────────┐
│ total_rating │
│    int128    │
├──────────────┤
│       333263 │
└──────────────┘

In [3]:
duckdb.sql("""
    WITH RECURSIVE input AS (
        SELECT '%s' AS input
    ), workflows AS (
        SELECT SUBSTRING(wf, 1, INSTR(wf, '{')-1) wf_name, STRING_SPLIT(REGEXP_EXTRACT(wf, '\\{([^}]*)\\}', 1), ',') wf_route
        FROM (
            SELECT UNNEST(STRING_SPLIT(SPLIT_PART(input, '\n\n', 1), '\n')) wf
            FROM input
        )
    ), parts AS (
        SELECT
            ROW_NUMBER() OVER() part_no,
            CAST(REGEXP_EXTRACT(part, 'x=([0-9]+)', 1) AS INT) AS x,
            CAST(REGEXP_EXTRACT(part, 'm=([0-9]+)', 1) AS INT) AS m,
            CAST(REGEXP_EXTRACT(part, 'a=([0-9]+)', 1) AS INT) AS a,
            CAST(REGEXP_EXTRACT(part, 's=([0-9]+)', 1) AS INT) AS s
        FROM (
            SELECT UNNEST(STRING_SPLIT(SPLIT_PART(input, '\n\n', 2), '\n')) part
            FROM input
        )
    ), routes AS (
        SELECT wf_name, route_idx, SUBSTRING(cond, 1, 1) xmas, SUBSTRING(cond, 2, 1) op, CAST(SUBSTRING(cond, 3) AS INT) val, tgt
        FROM (
            SELECT wf_name, route_idx, NULLIF(REGEXP_EXTRACT(route, '([^:]+):', 1), '') cond, SPLIT_PART(route, ':', -1) tgt
            FROM (
                SELECT wf_name, GENERATE_SUBSCRIPTS(wf_route, 1) AS route_idx, UNNEST(wf_route) route
                FROM workflows
            )
        )
    ), init_ranges AS (
        SELECT
            {'min': 1, 'max': 4000} x,
            {'min': 1, 'max': 4000} m,
            {'min': 1, 'max': 4000} a,
            {'min': 1, 'max': 4000} s
    ), sequences AS (
        SELECT
            CASE WHEN r.xmas = 'x' THEN
                CASE
                    WHEN r.op = '>' THEN UNNEST(ARRAY[{'min': r.val+1, 'max': i.x.max}, {'min': i.x.min, 'max': r.val}])
                    WHEN r.op = '<' THEN UNNEST(ARRAY[{'min': i.x.min, 'max': r.val-1}, {'min': r.val, 'max': i.x.max}])
                END
            ELSE i.x END x,
            CASE WHEN r.xmas = 'm' THEN
                CASE
                    WHEN r.op = '>' THEN UNNEST(ARRAY[{'min': r.val+1, 'max': i.m.max}, {'min': i.m.min, 'max': r.val}])
                    WHEN r.op = '<' THEN UNNEST(ARRAY[{'min': i.m.min, 'max': r.val-1}, {'min': r.val, 'max': i.m.max}])
                END
            ELSE i.m END m,
            CASE WHEN r.xmas = 'a' THEN
                CASE
                    WHEN r.op = '>' THEN UNNEST(ARRAY[{'min': r.val+1, 'max': i.a.max}, {'min': i.x.min, 'max': r.val}])
                    WHEN r.op = '<' THEN UNNEST(ARRAY[{'min': i.a.min, 'max': r.val-1}, {'min': r.val, 'max': i.x.max}])
                END
            ELSE i.a END a,
            CASE WHEN r.xmas = 's' THEN
                CASE
                    WHEN r.op = '>' THEN UNNEST(ARRAY[{'min': r.val+1, 'max': i.s.max}, {'min': i.x.min, 'max': r.val}])
                    WHEN r.op = '<' THEN UNNEST(ARRAY[{'min': i.s.min, 'max': r.val-1}, {'min': r.val, 'max': i.x.max}])
                END
            ELSE i.s END s,
            CASE WHEN r.xmas IN ('x', 'm', 'a', 's') THEN UNNEST(ARRAY[{'tgt': r.tgt, 'idx': 1}, {'tgt': r.wf_name, 'idx': r.route_idx + 1}]) ELSE {'tgt': r.tgt, 'idx': 1} END result
        FROM init_ranges i, routes r
        WHERE r.wf_name = 'in'
        AND r.route_idx = 1
        UNION ALL
        SELECT x, m, a, s, result
        FROM (
            SELECT
                CASE WHEN r.xmas = 'x' AND r.val BETWEEN i.x.min AND i.x.max THEN
                    CASE
                        WHEN r.op = '>' THEN UNNEST(ARRAY[{'min': r.val+1, 'max': i.x.max}, {'min': i.x.min, 'max': r.val}])
                        WHEN r.op = '<' THEN UNNEST(ARRAY[{'min': i.x.min, 'max': r.val-1}, {'min': r.val, 'max': i.x.max}])
                    END
                WHEN r.xmas = 'x' AND (r.op = '>' AND r.val > i.x.min) THEN NULL
                WHEN r.xmas = 'x' AND (r.op = '<' AND r.val < i.x.min) THEN NULL
                ELSE i.x END x,
                CASE WHEN r.xmas = 'm' AND r.val BETWEEN i.m.min AND i.m.max THEN
                    CASE
                        WHEN r.op = '>' THEN UNNEST(ARRAY[{'min': r.val+1, 'max': i.m.max}, {'min': i.m.min, 'max': r.val}])
                        WHEN r.op = '<' THEN UNNEST(ARRAY[{'min': i.m.min, 'max': r.val-1}, {'min': r.val, 'max': i.m.max}])
                    END
                WHEN r.xmas = 'm' AND (r.op = '>' AND r.val > i.m.min) THEN NULL
                WHEN r.xmas = 'm' AND (r.op = '<' AND r.val < i.m.min) THEN NULL
                ELSE i.m END m,
                CASE WHEN r.xmas = 'a' AND r.val BETWEEN i.a.min AND i.a.max THEN
                    CASE
                        WHEN r.op = '>' THEN UNNEST(ARRAY[{'min': r.val+1, 'max': i.a.max}, {'min': i.a.min, 'max': r.val}])
                        WHEN r.op = '<' THEN UNNEST(ARRAY[{'min': i.a.min, 'max': r.val-1}, {'min': r.val, 'max': i.a.max}])
                    END
                WHEN r.xmas = 'a' AND (r.op = '>' AND r.val > i.a.min) THEN NULL
                WHEN r.xmas = 'a' AND (r.op = '<' AND r.val < i.a.min) THEN NULL
                ELSE i.a END a,
                CASE WHEN r.xmas = 's' AND r.val BETWEEN i.s.min AND i.s.max THEN
                    CASE
                        WHEN r.op = '>' THEN UNNEST(ARRAY[{'min': r.val+1, 'max': i.s.max}, {'min': i.s.min, 'max': r.val}])
                        WHEN r.op = '<' THEN UNNEST(ARRAY[{'min': i.s.min, 'max': r.val-1}, {'min': r.val, 'max': i.s.max}])
                    END
                WHEN r.xmas = 's' AND (r.op = '>' AND r.val > i.s.min) THEN NULL
                WHEN r.xmas = 's' AND (r.op = '<' AND r.val < i.s.min) THEN NULL
                ELSE i.s END s,
                CASE
                    WHEN r.xmas IN ('x', 'm', 'a', 's') THEN UNNEST(ARRAY[{'tgt': r.tgt, 'idx': 1}, {'tgt': r.wf_name, 'idx': r.route_idx + 1}])
                    ELSE {'tgt': r.tgt, 'idx': 1}
                END result
            FROM sequences i, routes r
            WHERE r.wf_name = i.result.tgt
            AND r.route_idx = i.result.idx
        )
        WHERE x IS NOT NULL
        AND m IS NOT NULL
        AND a IS NOT NULL
        AND s IS NOT NULL
    ), valid_ranges AS (
        SELECT DISTINCT x, m, a, s
        FROM sequences
        WHERE result.tgt = 'A'
    ), check_overlap AS (
        SELECT *
        FROM valid_ranges vr1, valid_ranges vr2
        WHERE vr1.x <> vr2.x
        AND vr1.m <> vr2.m
        AND vr1.a <> vr2.a
        AND vr1.s <> vr2.s
        AND (vr1.x.min BETWEEN vr2.x.min AND vr2.x.max OR vr1.x.max BETWEEN vr2.x.min AND vr2.x.max)
        AND (vr1.m.min BETWEEN vr2.m.min AND vr2.m.max OR vr1.m.max BETWEEN vr2.m.min AND vr2.m.max)
        AND (vr1.a.min BETWEEN vr2.a.min AND vr2.a.max OR vr1.a.max BETWEEN vr2.a.min AND vr2.a.max)
        AND (vr1.s.min BETWEEN vr2.s.min AND vr2.s.max OR vr1.s.max BETWEEN vr2.s.min AND vr2.s.max)
    )
    SELECT SUM(CAST(x.max - x.min + 1 AS BIGINT) * CAST(m.max - m.min + 1 AS BIGINT) * CAST(a.max - a.min + 1 AS BIGINT) * CAST(s.max - s.min + 1 AS BIGINT)) n
    FROM valid_ranges

""" % input)

┌─────────────────┐
│        n        │
│     int128      │
├─────────────────┤
│ 130745440937650 │
└─────────────────┘